In [3]:
import tensorflow as tf
import os  
import sys
from tensorflow import logging
from tensorflow import gfile
import pprint
import pickle
import numpy as np
import random
import math
from PIL import Image
from tensorflow.examples.tutorials.mnist import input_data

In [ ]:
# 准备数据集 Mnist

mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

output_dir = '.local_run'    
if not gfile.Exists(output_dir):
    gfile.MakeDirs(output_dir)
    
def get_default_params():
    return tf.contrib.training.HParams(
        z_dim = 100, # 输入向量size
        init_conv_size = 4, # 反卷积层size
        g_channels = [128, 64, 32, 1], # 生成器每一层反卷积的通道数
        d_channels = [32, 64, 128, 256], # 判别器中每一层卷积层的通道数
        batch_size = 128,
        learning_rate = 0.002,
        betal = 0.5, # adam参数
        img_size = 32, # 反卷积size为4x4，所以要经过3次反卷积
    )

hps = get_default_params()
print(hps.img_size)
print(hps.g_channels)
print(mnist.train.images.shape)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use urllib or similar directly.
